<a href="https://colab.research.google.com/github/Jahanvi-123/Jahanvi/blob/main/Student_Profile_Analyzer_Project_1_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Install required libraries
!pip install -q langchain langchain-google-genai langchain-community faiss-cpu pypdf sentence-transformers python-dotenv
# Install the fpdf library
!pip install fpdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 86.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/5

In [ ]:
import os
from google.colab import files, userdata
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime
import gradio as gr # Import gradio library

In [ ]:
# Step 3: Set up Google Gemini API key using Colab Secrets
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
except:
    print("Error: GOOGLE_API_KEY not found in Colab Secrets.")
    print("1. Go to the 'Secrets' tab in the left sidebar (key icon).")
    print("2. Add a new secret with name 'GOOGLE_API_KEY' and your Gemini API key as the value.")
    print("3. Get your API key from https://ai.google.dev/gemini-api/docs/api-key")
    raise Exception("GOOGLE_API_KEY not found")

In [ ]:
# Step 4: Initialize the Gemini model and embeddings
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GOOGLE_API_KEY)
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
# Step 5: Upload the student profile PDF
print("Please upload the student profile PDF file:")
uploaded = files.upload()
pdf_file = list(uploaded.keys())[0]

Please upload the student profile PDF file:


Saving Student_Profiles.pdf to Student_Profiles.pdf


In [ ]:
# Step 6: Load and process the PDF
loader = PyPDFLoader(pdf_file)
documents = loader.load()

In [ ]:
# Step 7: Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [ ]:
# Step 8: Create FAISS vector store
vectorstore = FAISS.from_documents(texts, embeddings)

In [ ]:
# Step 9: Set up the retriever
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [ ]:
# Step 10: Define the prompt template for cover letter generation
prompt_template = """
You are a professional career advisor. Based on the context extracted from the student's profile and the details below, generate a professional, personalized cover letter.

Extract student address, email, and other details from the context.

Inputs:
- Student Name: {student_name}
- Company Name: {company_name}
- Company Address: {company_address}
- Position: {position}
- Date: {current_date}

Context from student profile:
{context}

Format the cover letter formally with this structure:

{student_name}
[Student Address from profile]
[City, State, ZIP Code from profile]
[Email Address from profile]
{current_date}

Hiring Manager
{company_name}
{company_address}

Dear Hiring Manager,

[Personalized body based on context and position]

Sincerely,
{student_name}
"""
prompt = ChatPromptTemplate.from_template(prompt_template)

In [ ]:
# Step 11: Define a function to format retrieved documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# Step 12: Set up the RAG chain using LCEL
rag_chain = (
    {
        "context": (lambda x: x["student_name"]) | retriever | format_docs, # Extract student name, retrieve context, and format documents
        "student_name": (lambda x: x["student_name"]), # Pass student name directly
        "company_name": (lambda x: x["company_name"]), # Pass company name directly
        "company_address": (lambda x: x["company_address"]), # Pass company address directly
        "position": (lambda x: x["position"]), # Pass position directly
        "current_date": (lambda x: x["current_date"]) # Pass current date directly
    }
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# User inputs company info
student_name = input("Enter student name: ") # Add input for student name
company_name = input("Enter company name: ")
company_address = input("Enter company address: ")
position = input("Enter position: ")

# Current date
current_date = datetime.now().strftime("%B %d, %Y")

# Call the RAG chain
input_data = {
    "student_name": student_name, # Pass student name directly
    "company_name": company_name,
    "company_address": company_address,
    "position": position,
    "current_date": current_date
}

response = rag_chain.invoke(input_data)
print("\nGenerated Cover Letter:\n")
print(response)

Enter student name: Allison Hills
Enter company name: Amazon 
Enter company address: Hyderabad
Enter position: SDE Intern

Generated Cover Letter:

```
Allison Hills
65423 Garcia Light, West Melanieview, AZ 96148
jillrhodes@miller.com
July 04, 2025

Hiring Manager
Amazon 
Hyderabad

Dear Hiring Manager,

I am writing to express my enthusiastic interest in the SDE Intern position at Amazon, Hyderabad. As a B.Tech student in Civil Engineering at Herrera-Dudley, I possess a strong foundation in analytical thinking and problem-solving, skills I believe are readily transferable and highly valuable in a software development environment.

While my academic background is in Civil Engineering, I have actively pursued my passion for technology through self-study and hands-on projects. I have developed proficiency in Cloud Computing, Python, and Machine Learning, and I am eager to apply these skills to real-world challenges at Amazon. My interest in robotics, open source projects, and startups al

In [ ]:
# Step 14: Display the generated cover letter
print("\nGenerated Cover Letter:\n")
print(response)


Generated Cover Letter:

```
Allison Hills
65423 Garcia Light, West Melanieview, AZ 96148
jillrhodes@miller.com
July 04, 2025

Hiring Manager
Amazon 
Hyderabad

Dear Hiring Manager,

I am writing to express my enthusiastic interest in the SDE Intern position at Amazon, Hyderabad. As a B.Tech student in Civil Engineering at Herrera-Dudley, I possess a strong foundation in analytical thinking and problem-solving, skills I believe are readily transferable and highly valuable in a software development environment.

While my academic background is in Civil Engineering, I have actively pursued my passion for technology through self-study and hands-on projects. I have developed proficiency in Cloud Computing, Python, and Machine Learning, and I am eager to apply these skills to real-world challenges at Amazon. My interest in robotics, open source projects, and startups aligns perfectly with Amazon's innovative culture and commitment to pushing technological boundaries.

Although my CGPA is 7

In [ ]:
import gradio as gr
from fpdf import FPDF
import tempfile

# Dummy rag_chain.invoke implementation (replace with your actual logic)
# from your_existing_code import rag_chain

def generate_cover_letter_gradio(student_name_input, pdf_file_input, company_name_input, company_address_input, position_input, current_date_input):
    if pdf_file_input is None:
        return "Please upload a student profile PDF.", None

    # Load PDF text (this is mock - you can connect RAG here)
    # from PyPDF2 import PdfReader
    # reader = PdfReader(pdf_file_input.name)
    # text = "\n".join([page.extract_text() for page in reader.pages])

    # Use the actual RAG chain to generate the cover letter
    input_data = {
        "student_name": student_name_input,
        "company_name": company_name_input,
        "company_address": company_address_input,
        "position": position_input,
        "current_date": current_date_input if current_date_input else datetime.now().strftime("%B %d, %Y")
    }
    response = rag_chain.invoke(input_data)


    # Convert response to PDF
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.set_font("Arial", size=12)
    for line in response.split("\n"):
        pdf.multi_cell(0, 10, line)

    # Save to temp
    clean_name = student_name_input.replace(" ", "_").replace("/", "_")
    file_path = os.path.join(tempfile.gettempdir(), f"Cover_Letter_{clean_name}.pdf")
    pdf.output(file_path)

    return response, file_path


# ------------------ CSS Styling ------------------
custom_css = """
.gradio-container {
    max-width: 900px !important;
    margin: 0 auto !important;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%) !important;
    min-height: 100vh !important;
    padding: 20px !important;
}
.custom-title {
    text-align: center !important;
    color: #800080 !important;
    font-size: 2.5rem !important;
    font-weight: 700 !important;
    margin-bottom: 10px !important;
}
.gradio-container p {
    color: rgba(255,255,255,0.9) !important;
    text-align: center !important;
    font-size: 1.1rem !important;
    margin-bottom: 30px !important;
    font-weight: 300 !important;
}
.input-container {
    background: #f8f9ff !important;
    border-radius: 15px !important;
    padding: 25px !important;
    margin-bottom: 20px !important;
    border: 2px solid #e1e5f2 !important;
}
.gr-textbox {
    border-radius: 25px !important;
    border: 2px solid #d1d9ff !important;
    padding: 15px 20px !important;
    font-size: 16px !important;
    background: white !important;
}
.gr-textbox:focus {
    border-color: #667eea !important;
    box-shadow: 0 0 0 3px rgba(102, 126, 234, 0.1) !important;
    outline: none !important;
}
.gr-file {
    border-radius: 15px !important;
    border: 2px dashed #d1d9ff !important;
    padding: 20px !important;
    background: white !important;
}
.output-container {
    background: white !important;
    border-radius: 15px !important;
    padding: 20px !important;
    margin-top: 20px !important;
    box-shadow: 0 4px 12px rgba(0,0,0,0.1) !important;
    position: relative !important;
}
.output-container::before {
    content: '🤖' !important;
    position: absolute !important;
    top: -10px !important;
    left: 20px !important;
    background: #667eea !important;
    color: white !important;
    padding: 8px 12px !important;
    border-radius: 20px !important;
    font-size: 16px !important;
}
::-webkit-scrollbar {
    width: 8px !important;
}
::-webkit-scrollbar-track {
    background: #f1f1f1 !important;
    border-radius: 10px !important;
}
::-webkit-scrollbar-thumb {
    background: #667eea !important;
    border-radius: 10px !important;
}
::-webkit-scrollbar-thumb:hover {
    background: #764ba2 !important;
}
"""

# ------------------ Gradio UI ------------------
with gr.Blocks(css=custom_css, theme=gr.themes.Soft(
    primary_hue="blue",
    secondary_hue="purple",
    neutral_hue="slate",
    font=gr.themes.GoogleFont("Inter")
)) as interface:

    # Title and subtitle
    gr.HTML("""
    <div class='custom-title'>🤖 AI Cover Letter Generator</div>
    <p>💬 Upload your student profile and enter a name to generate a professional PDF cover letter.</p>
    """)

    with gr.Row():
        with gr.Column():
            student_name_input = gr.Textbox(
                label="👤 Student Name",
                placeholder="Enter the student name (e.g., Emily Rios)",
                elem_classes=["input-container"]
            )
            pdf_upload_input = gr.File(
                label="📄 Upload Student PDF",
                file_types=[".pdf"],
                elem_classes=["input-container"]
            )
            company_name_input = gr.Textbox(
                label="🏢 Company Name",
                placeholder="Enter the company name (e.g., Google)",
                elem_classes=["input-container"]
            )
            company_address_input = gr.Textbox(
                label="🏠 Company Address",
                placeholder="Enter the company address",
                elem_classes=["input-container"])


            position_input = gr.Textbox( # Corrected variable name
                label="💼 Position Title",
                placeholder="Enter the position (e.g., Data Analyst Intern)",
                elem_classes=["input-container"]
            )
            current_date_input = gr.Textbox(
                label="📅 Date (optional)",
                placeholder="Leave blank to use today’s date",
                elem_classes=["input-container"])
            generate_btn = gr.Button("Generate Cover Letter")

        with gr.Column():
            output_box = gr.Textbox(
                lines=20,
                label="🤖 AI Generated Cover Letter",
                elem_classes=["output-container"]
            )
            download_button = gr.File(label="📥 Download Cover Letter (PDF)")

    # Connect button to function
    generate_btn.click(
        fn=generate_cover_letter_gradio,
        inputs=[student_name_input, pdf_upload_input, company_name_input, company_address_input, position_input, current_date_input], # Use correct variable names
        outputs=[output_box, download_button]
    )

# Launch the app
interface.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://bb373012fa6a4bd347.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
